In [ ]:

!pip install gensim
!pip install farasa
!pip install FarasaSegmenter
!pip install pyarabic
!pip install arabicstopwords
!pip install stop-words
!pip install tashaphyne
!pip install Arabic-Stopwords
!pip install farasapy
!pip install nltk
import nltk

# Download the 'punkt' resource
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 10.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement FarasaSegmenter (from versions: none)
ERROR: No matching distribution found for FarasaSegmenter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement arabicstopwords (from versions: none)
ERROR: No matching distribution found for arabicstopwords
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=d3e088692aa14a0f9dc18a825a85cfd5113fbba6b914dec200463fb1c9d770a1
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 6.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import RegexpTokenizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
from pyarabic import trans
import arabicstopwords.arabicstopwords as stp
import string
from itertools import groupby
import re
from gensim import corpora, models


import http.client
import json
import requests
from tashaphyne.stemming import ArabicLightStemmer

#Strip vowels from a text, include Shadda
from pyarabic.araby import strip_tashkeel
#Standardize the Hamzat into one form of hamza, replace Madda by hamza and alef.
import pyarabic.araby as araby

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class preprocessing:

  arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
  english_punctuations = string.punctuation
  punctuations_list = arabic_punctuations + english_punctuations
  stop_words = stopwords.words('arabic')
  stop_words.append(['ان','او','الى','اي','اما','شي','الا','يا'])
  ArListem = ArabicLightStemmer()

  def __init__(self, stemming = None, del_punctiation = True, del_emoji = True, del_consecutive = True, del_stopword = True):
    self.stemming = stemming
    self.del_punctiation = del_punctiation
    self.del_emoji = del_emoji
    self.del_consecutive = del_consecutive
    self.del_stopword = del_stopword
    self.indexLoop = 0


  #Flatten
  def __flatten(self, t):
    return [item for sublist in t for item in sublist]

  #remove punctiation
  def __remove_punctuations(self, text):
    translator = str.maketrans('', '', self.punctuations_list)
    return text.translate(translator)

  def del_punc(self, elements):
    if self.del_punctiation == True:
      wordsForTf = []
      for element in elements:
        word = self.__remove_punctuations(element)
        if word != '':
          wordsForTf.append(word)
      return wordsForTf
    else:
      return elements

  #remove consecutive
  def remove_consecutive(self, text:list):
    if self.del_consecutive == True:
      comment=list()
      for word in text:
          if len(word)==1:
              continue
          groups = groupby(word)
          new_text=''
          result = [(label, sum(1 for _ in group)) for label, group in groups]
          for letter in result:
              if letter[1]>2:
                  new_text+=letter[0]
              else:
                  if letter[1]==2:
                      for i in range(2):
                          new_text+=letter[0]
                  else:
                      new_text+=letter[0]
          comment.append(new_text)
      return comment
    else:
      return text

  #remove Emoji
  def deEmoji(self, text):
    if self.del_emoji == True:
      regrex_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"
          u"\U0001F300-\U0001F5FF"
          u"\U0001F680-\U0001F6FF"
          u"\U0001F1E0-\U0001F1FF"
                            "]+", flags = re.UNICODE)
      return regrex_pattern.sub(r'',text)
    else:
      return text

  #remove Stop Words
  def del_StopWords(self, wordss,hamza_normalize=False):

    if self.del_stopword == True:
      if hamza_normalize == True:
        stop_words_normalize=[self.normalize_hamza(word) for word in self.stop_words]
        return [ [word for word in comment if word not in stop_words_normalize] for comment in wordss ]

      wordsWstop = []
      wordsWstopT = []
      for liste in wordss:
        wordsWstop.append([element for element in liste if stp.is_stop(element) == False])

      for liste in wordsWstop:
        wordsWstopT.append([element for element in liste if element not in self.stop_words])
      return wordsWstopT
    else:
      return wordss


  def token(self, textSEmoji):
    words = []
    for comment in textSEmoji:
      words.append(tokenize(comment, conditions=is_arabicrange, morphs=strip_tashkeel))

    return words


  def stemLineFarasa(self, tokens):
    text = " ".join(tokens)
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    payload = {"text": text}
    headers = {"content-type": "application/json", "cache-control": "no-cache"}
    conn.request("POST", "/msa/webapi/lemma", json.dumps(payload), headers, encode_chunked=False)
    res = conn.getresponse()
    data = res.read()
    result = json.loads(data.decode("utf-8"))
    print(self.indexLoop)
    self.indexLoop += 1
    return result["result"]


  def stemLineISRI(self, tokens):
    isri = ISRIStemmer()
    StemISRI = [isri.stem(text) for text in tokens]
    return StemISRI

  def stemLineLight(self, tokens):
    StemLight = []
    for token in tokens:
      stem = self.ArListem.light_stem(token)
      StemLight.append(self.ArListem.get_stem())
    return StemLight


  def root(self, tokens):
    racine = []
    for token in tokens:
      stem = self.ArListem.light_stem(token)
      racine.append(self.ArListem.get_root())
    return racine

  def uniques(self,data):
        #finding unique
    unique = []
    for doc in data:
      for word in doc:
        if word not in unique:
            unique.append(word)
    return len(unique)


  def remove_length2(self,data:list,length=2):
    return [[word for word in comment if len(word) >length ] for comment in data]


  def normalize_hamza(self,data,method="tasheel"):
    if type(data)==str:
      return araby.normalize_hamza(data, method=method)

    return [araby.normalize_hamza(comment, method=method) for comment in data]

  def preprocess(self, data):
    #list of text data
    textSEmoji = np.array([self.deEmoji(text) for text in data])

    wordss = []
    for word in self.token(textSEmoji):
      # word is vector of words
      wordss.append(self.remove_consecutive(word))

    wordss=self.del_StopWords(wordss)

    if self.stemming == 'farasa':
      stemmi = [self.stemLineFarasa(tokens) for tokens in wordss]
    elif self.stemming == 'isri':
      stemmi = [self.stemLineISRI(tokens) for tokens in wordss]
    elif self.stemming == 'light':
      stemmi = [self.stemLineLight(tokens) for tokens in wordss]
    elif self.stemming == 'root':
      stemmi = [self.root(tokens) for tokens in wordss]
    else:
      stemmi = wordss

    wordsWstop = self.del_StopWords(stemmi)

    wordsForTf = np.array([self.del_punc(word) for word in wordsWstop])

    return wordsForTf

load the data

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/train.csv')
IDs=data['ID'].to_numpy()
comments = data['comment'].to_numpy()
labeles =  data['label'].to_numpy()



data_test = pd.read_csv('test_stage1.csv')
IDs_test=data_test['ID'].to_numpy()
comments_test = data_test['comment'].to_numpy()
data_test.head()


,ID,comment
0,1,"التلقيح نعمة لانه فابور,الناس في مصر را كتخلص ..."
1,2,أثبت التلقيح أهميته في الحد من الوفيات
2,3,انا ملقح جوج مرات والحمد لله عندي المناعة ديال...
3,4,كنــا متأكديــن من أن جلالـة الملــك سيجعل الت...
4,5,شعب أناني مشبع بثقافة الخرافة والمؤامرة… بنادم...


In [ ]:
import pandas as pd
from textblob import TextBlob  # For sentiment analysis

# Assuming you have a DataFrame called 'data' with a column 'comment'
# Replace 'data' with your actual DataFrame name

# Filter out short comments
data = data[data['comment'].apply(lambda x: len(str(x).split()) > 3)]

# Remove duplicate comments
data = data.drop_duplicates(subset=['comment'])

# Use sentiment analysis to filter out extremely negative or positive comments
data['sentiment'] = data['comment'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
data = data[data['sentiment'].between(-0.5, 0.5)]

# Drop the temporary 'sentiment' column
data = data.drop(columns=['sentiment'])

# Optionally, you can save the cleaned data to a new CSV file
data.to_csv('cleaned_data.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/cleaned_data.csv')
IDs=data['ID'].to_numpy()
comments = data['comment'].to_numpy()
labeles =  data['label'].to_numpy()

In [ ]:
prepro = preprocessing(stemming='light')
textPre = prepro.preprocess(comments)

textPre_dev = prepro.preprocess(comments_test)

train_input_data=[' '.join(doc) for doc in textPre]
test_input_data=[' '.join(doc) for doc in textPre_dev]

print(len(labeles))

<ipython-input-3-43071c50821a>:182: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  wordsForTf = np.array([self.del_punc(word) for word in wordsWstop])


1898


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import joblib

# Assuming you have already loaded your data
# ...

# Perform train-test split on the training data
train_input_data, validation_input_data, labels_train, labels_validation = train_test_split(
    train_input_data, labeles, test_size=0.2, random_state=42
)

# Tokenize and pad the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_input_data)
train_sequences = tokenizer.texts_to_sequences(train_input_data)
validation_sequences = tokenizer.texts_to_sequences(validation_input_data)

max_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in validation_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding='post')

# Define and train the embedding model with different optimizers
optimizers = ['adam', 'sgd']  # Add more optimizers if needed

for optimizer_name in optimizers:
    print(f"Training with optimizer: {optimizer_name}")

    # Create a new model for each optimizer
    model = Sequential([
        Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model with the selected optimizer
    if optimizer_name.lower() == 'adam':
        optimizer = Adam()
    elif optimizer_name.lower() == 'sgd':
        optimizer = SGD()
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model and validate on the validation set
    model.fit(
        train_padded, labels_train,
        epochs=5,
        validation_data=(validation_padded, labels_validation)
    )

    # Save the trained model
    model.save(f'/content/your_trained_model_{optimizer_name}.h5')

    # Extract features using the trained model
    embedding_model = Sequential([
        model.layers[0]  # Assuming the embedding layer is the first layer in your model
    ])

    train_features = embedding_model.predict(train_padded)
    train_features_flattened = train_features.reshape(train_features.shape[0], -1)

    # Train the SVM model
    svm_model = SVC()
    svm_model.fit(train_features_flattened, labels_train)

    # Save the SVM model
    joblib.dump(svm_model, f'/content/svm_model_{optimizer_name}.joblib')

    # Continue with the rest of your code...

# Optionally, print the final validation accuracy after training
final_validation_accuracy = history.history['val_accuracy'][-1]
print(f'Final Validation Accuracy: {final_validation_accuracy * 100:.2f}%')


Training with optimizer: adam
Epoch 1/5
48/48 [==============================] - 3s 42ms/step - loss: 0.6763 - accuracy: 0.5817 - val_loss: 0.6352 - val_accuracy: 0.6605
Epoch 2/5
48/48 [==============================] - 1s 27ms/step - loss: 0.4887 - accuracy: 0.7958 - val_loss: 0.5323 - val_accuracy: 0.7263
Epoch 3/5
48/48 [==============================] - 1s 27ms/step - loss: 0.1326 - accuracy: 0.9651 - val_loss: 0.5022 - val_accuracy: 0.7553
Epoch 4/5
48/48 [==============================] - 1s 26ms/step - loss: 0.0244 - accuracy: 0.9974 - val_loss: 0.5352 - val_accuracy: 0.7421
Epoch 5/5
48/48 [==============================] - 0s 4ms/step
Training with optimizer: sgd
Epoch 1/5
48/48 [==============================] - 3s 40ms/step - loss: 0.6930 - accuracy: 0.5132 - val_loss: 0.6908 - val_accuracy: 0.5684
Epoch 2/5
48/48 [==============================] - 1s 27ms/step - loss: 0.6904 - accuracy: 0.5922 - val_loss: 0.6879 - val_accuracy: 0.6263
Epoch 3/5
48/48 [=====================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


48/48 [==============================] - 0s 3ms/step
Final Validation Accuracy: 76.58%


In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained embedding model
embedding_model = load_model('/content/your_trained_model_adam.h5')


# Tokenize and pad the test data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_input_data)
test_sequences = tokenizer.texts_to_sequences(test_input_data)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Extract features using the trained embedding model
test_features = embedding_model.predict(test_padded)
test_features_flattened = test_features.reshape(test_features.shape[0], -1)

# Load the labels or ground truth for the test data
labels_test = data_test['label'].to_numpy()

# Train a Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(test_features_flattened, labels_test)

# Optionally, you can print the accuracy on the training set (although it's typically more meaningful on a separate test set)
train_predictions = nb_model.predict(test_features_flattened)
train_accuracy = accuracy_score(labels_test, train_predictions)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')



In [ ]:
 import pandas as pd
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
import joblib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the saved models
optimizers = ['adam', 'sgd']

for optimizer_name in optimizers:
    # Load the trained model
    model = load_model(f'/content/your_trained_model_adam.h5')

    # Load the SVM model
    svm_model = joblib.load(f'/content/svm_model_{optimizer_name}.joblib')

    # Load the Naive Bayes model
    nb_model = joblib.load(f'/content/nb_model_{optimizer_name}.joblib')

    # Load the test data
    data_test = pd.read_excel('/content/Dataset.xlsx')
    comments_test = data_test['comment'].to_numpy()

    # Preprocess the test data
    prepro = preprocessing(stemming='light')
    textPre_test = prepro.preprocess(comments_test)
    test_input_data = [' '.join(doc) for doc in textPre_test]

    # Tokenize and pad the test data
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_input_data)
    test_sequences = tokenizer.texts_to_sequences(test_input_data)
    test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

    # Extract features using the trained model
    embedding_model = Sequential([model.layers[0]])

    test_features = embedding_model.predict(test_padded)
    test_features_flattened = test_features.reshape(test_features.shape[0], -1)

    # Predict using SVM
    svm_predictions = svm_model.predict(test_features_flattened)

    # Predict using Naive Bayes
    nb_predictions = nb_model.predict(test_features_flattened)

    # Print SVM accuracy
    svm_accuracy = accuracy_score(labels_test, svm_predictions)
    print(f'SVM Accuracy with {optimizer_name} optimizer: {svm_accuracy * 100:.2f}%')

    # Print Naive Bayes accuracy
    nb_accuracy = accuracy_score(labels_test, nb_predictions)
    print(f'Naive Bayes Accuracy with {optimizer_name} optimizer: {nb_accuracy * 100:.2f}%')


In [ ]:
# Assuming the previous code has been executed

# Load the test data
excel_file_path = '/content/2 vaccination train set.xlsx'
data_test = pd.read_excel(excel_file_path)
comments_test = data_test['comment'].to_numpy()
labels_test = data_test['labels'].to_numpy()

# Preprocess the test data
textPre_test = prepro.preprocess(comments_test)
test_input_data = [' '.join(doc) for doc in textPre_test]

# Tokenize and pad the test data
test_sequences = tokenizer.texts_to_sequences(test_input_data)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Evaluate the models on the test set
for optimizer_name in optimizers:
    print(f"Evaluating model with optimizer: {optimizer_name}")

    # Load the trained embedding model
    trained_model = load_model(f'/content/your_trained_model_adam.h5')

    # Extract features using the trained embedding model
    embedding_model = Sequential([
        trained_model.layers[0]  # Assuming the embedding layer is the first layer in your model
    ])

    test_features = embedding_model.predict(test_padded)
    test_features_flattened = test_features.reshape(test_features.shape[0], -1)

    # Load the trained SVM model
    loaded_svm_model = joblib.load(f'/content/svm_model_adam.joblib')

    # Predict labels for the test data using SVM
    test_predictions = loaded_svm_model.predict(test_features_flattened)

    # Calculate and print accuracy
    test_accuracy = accuracy_score(labels_test, test_predictions)
    print(f'Test Accuracy with {optimizer_name} optimizer: {test_accuracy * 100:.2f}%')


<ipython-input-3-43071c50821a>:182: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  wordsForTf = np.array([self.del_punc(word) for word in wordsWstop])


Evaluating model with optimizer: adam
45/45 [==============================] - 0s 2ms/step
Test Accuracy with adam optimizer: 94.38%
Evaluating model with optimizer: sgd
45/45 [==============================] - 0s 2ms/step
Test Accuracy with sgd optimizer: 94.38%
